## ABC applied to position reconstruction

*Bart Pelssers, 26-02-2018*

This notebook provides some ingredients for applying the ABC algorithm to position reconstruction, the most basic case. Just to test the framework.


* Provides:
  * prior mean
  * forward model
  * summary statistic
  
*Umberto Simola, 22-03-2018*

Provided the ABC-PMC algorithm for running the analyses.

In [2]:
import numpy as np

In [3]:
# Using the XENON1T data processor
from pax import utils
from pax.configuration import load_configuration
from pax.PatternFitter import PatternFitter
from pax.plugins.io.WaveformSimulator import uniform_circle_rv

In [4]:
class PriorPosition():
    """Implements the calculation of the mean of a prior
       given a pattern (either from data or from the forward model)
    """
    
    def __init__(self):
        # Get some settings from the XENON1T detector configuration
        config = load_configuration('XENON1T')
        
        # PMT positions
        pmt_config = config['DEFAULT']['pmts']
        
        # List of dicts {'x': , 'y'}, which the position
        self.positions = [pmt['position'] for pmt in pmt_config][:127]

    def __call__(self, pattern):
        # The id of the PMT that sees most light
        max_pmt = np.argmax(pattern)
        
        # The position of that PMT
        pos = self.positions[max_pmt]
        
        return pos['x'], pos['y']        
        

class Model():
    """Implements the forward model for ABC project.
    
       The forward model used here is the most basic test case.
       It draws from the per-PMT S2 LCE maps to provide a
       hitpattern for a given x,y. Assumes all top PMTs live.
       Also the total number of detected photo-electrons needs to
       be specified, this is set constant by default.
    """

    def __init__(self):
        # Get some settings from the XENON1T detector configuration
        config = load_configuration('XENON1T')
        
        # The per-PMT S2 LCE maps (and zoom factor which is a technical detail)
        lce_maps = config['WaveformSimulator']['s2_patterns_file']
        lce_map_zoom = config['WaveformSimulator']['s2_patterns_zoom_factor']

        # Simulate the right PMT response
        qes = np.array(config['DEFAULT']['quantum_efficiencies'])
        top_pmts = config['DEFAULT']['channels_top']
        errors = config['DEFAULT']['relative_qe_error'] + config['DEFAULT']['relative_gain_error']

        # Set up the PatternFitter which sample the LCE maps
        self.pf = PatternFitter(filename=utils.data_file_name(lce_maps),
                                zoom_factor=2*lce_map_zoom,
                                adjust_to_qe=qes[top_pmts],
                                default_errors=errors)

    def __call__(self, x, y, n_obs = 500):
        """Returns a hitpattern of n_obs photo-electrons
           for given x, y position.
        """
        
        return n_obs * self.pf.expected_pattern((x, y))


class Generator():
    """Generates test hitpatterns by drawing from LCE maps (forward model)"""
    
    def __init__(self, model):
        # Get some settings from the XENON1T detector configuration
        config = load_configuration('XENON1T')
        self.tpc_radius = config['DEFAULT']['tpc_radius']
        
        # Use the forward model also as generator for now
        self.model = model
    
    def __call__(self):
        return self.model(*uniform_circle_rv(self.tpc_radius))

In [5]:
# Setup the Models
model = Model()
prior_mean = PriorPosition()
generator = Generator(model)

In [6]:
# Example pattern from x=2.6264, y=-17.96082

# The range of x and y is [-47.884375 cm, -47.884375 cm]
# But x**2 + y**2 < 47.884375**2
# Otherwise m() will raise and exception

pattern = model(2.6264, -17.96082)

print("Length of pattern: %d, Sum of pattern: %.2f" % (len(pattern), pattern.sum()))
print(pattern)

Length of pattern: 127, Sum of pattern: 500.00
[   0.41645266    0.38856616    0.40444191    0.32748224    0.38441824
    0.38170297    0.40477179    0.39144921    0.47687253    0.46203223
    0.52646186    0.57781957    0.64051689    0.74521969    0.84447167
    0.92923987    1.16943785    1.26806905    1.6059695     1.50876463
    1.39647011    1.18050349    1.1130101     0.94818458    0.87288815
    0.65681059    0.63291142    0.55729029    0.49757721    0.54446233
    0.47297285    0.43356499    0.33506074    0.46603606    0.43350709
    0.41858736    0.66362616    0.68610503    0.79095441    0.69480663
    0.75740472    0.74267691    0.70961981    0.92924549    0.84056631
    0.99494579    1.07601729    1.24895687    1.57851557    1.80697221
    2.25402829    2.67966085    2.88123522    2.19200447    1.84082469
    1.44009707    1.25218986    0.97101662    0.89157513    0.76710311
    0.73912278    0.6512003     0.68037602    0.69750597    0.78879756
    0.64265046    0.80342745  

In [7]:
# If using a 2D Normal prior this would be a good guess for the mean (x,y) of that prior.
prior_mean(pattern)

(4.115222817130077, -15.358220637996187)

In [8]:
###Function for sampling proposal coordinates from the prior: Normal Centered on the naiveCoord, sd=15
def priorFunction(coord):
    coordX = coord[0]
    coordY = coord[1]
    xProp=np.random.normal(coordX,15,1)[0]
    yProp=np.random.normal(coordY,15,1)[0]
    while xProp**2+yProp**2>47.884375**2:
        xProp=np.random.normal(coordX,15,1)[0]
        yProp=np.random.normal(coordY,15,1)[0]
    return xProp,yProp
#priorFunction(naiveCoord)

In [9]:
###Distance function for comparing the real and the simulated dataset
def rho(x,y):
    return sum((x-y)**2)/np.shape(x)[0]
    #return sum(np.abs(x-y))/np.shape(x)[0]
#print(rho(event,SimulatedModel))

In [10]:
###Load the true position matrix for generating the Observed dataset

###For sure you can load only the positions from the original files but I got errors when trying to call the data

trueCoordMatrix=np.loadtxt('truepos')
#print(trueCoordMatrix[0])
#np.shape(trueCoordMatrix)

In [ ]:
###ABC-SMC Definition of the necessary quantities

In [11]:
###Transformation kernel for resampling for t>0 (i.e. rather the using the prior we use this kernel)
def transfKernel(propx0,propy0,varx,vary):
    propx=np.random.normal(propx0,2*varx,1)[0]
    propy=np.random.normal(propy0,2*vary,1)[0]
    while propx**2+propy**2 > 47.884375**2:
        propx=np.random.normal(propx0,2*varx,1)[0]
        propy=np.random.normal(propy0,2*vary,1)[0]
    return propx,propy 

In [34]:
###ABC-SMC on a single selected event event j

In [16]:
import scipy.stats

###Number of particles for recunstructing the posterior for each coordinate
N=5000

###Number of iterations before stopping the algorithm
nIter=40

###Quantile used for shrinking the tolerances through the iterations
quantile=0.85

###Importance weigths for the ABC-PMC: because for t>0 we using a kernel and not the prior as proposal distribution
weights=np.zeros((nIter,N))
weights[0,:]=1/N

###Accepted elements are stored here
###abcCooord x
abcCoordsx=np.zeros((nIter,N))
###abcCooord y
abcCoordsy=np.zeros((nIter,N))

###Distance of accepted elements are stored here
d=np.zeros((nIter,N))

###First tolerance epsilon1
epsilon=4

###Total number of draws required for covering the entire analyses
totDraws=0

#########################################################################
#########################################################################
###pick the event j
j=0

###given an event, we need the true coordinates of this event and getting the 'true' dataset. The trueDataset called here
###TrueModel
trueCoord = trueCoordMatrix[j] 
TrueModel=model(trueCoord[0], trueCoord[1], 500)
###means of the priors
naiveCoord=prior_mean(TrueModel)

###Simulated accepted dataset from the last iteration
simAccData=np.zeros((N,np.shape(TrueModel)[0]))
    
for t in range(0,nIter):
    print(t)
    if(t==0):
        for i in range(0,N):
            d[t,i]=epsilon+1
            while d[t,i] > epsilon: 
                propCoord = priorFunction(naiveCoord)
                ###Deterministic FM
                #simulatedData = model(propCoord[0], propCoord[1], 500)
                ###Stochastic FM, for now adding gaussian noise (mean=0,sd=1)
                simulatedData = model(propCoord[0], propCoord[1], 500)+np.random.normal(0,1,np.shape(TrueModel)[0])
                totDraws = totDraws+1
                d[t,i] = rho(TrueModel,simulatedData) 
            abcCoordsx[t,i]=propCoord[0]
            abcCoordsy[t,i]=propCoord[1]
    else:
        epsilon= np.percentile(d[t-1,],quantile*100)
        print(epsilon)
        meanx=np.sum(abcCoordsx[t-1,:]*weights[t-1,:])
        varx=np.sum((abcCoordsx[t-1,:]-meanx)**2*weights[t-1,:])
        meany=np.sum(abcCoordsy[t-1,:]*weights[t-1,:])
        vary=np.sum((abcCoordsy[t-1,:]-meany)**2*weights[t-1,:])
        for i in range(0,N):
            d[t,i]=epsilon+1
            while d[t,i] > epsilon: 
                sample=np.random.choice(N,1,p=weights[t-1,:])
                propx0=abcCoordsx[t-1,sample]
                propy0=abcCoordsy[t-1,sample]
                prop=transfKernel(propx0,propy0,varx,vary)
                propx=prop[0]
                propy=prop[1]
                ###Deterministic FM
                #simulatedData = model(propx, propy, 500)
                ###Stochastic FM, for now adding gaussian noise (mean=0,sd=1)
                simulatedData = model(propx, propy, 500)+np.random.normal(0,1,np.shape(TrueModel)[0])
                totDraws = totDraws+1
                d[t,i] = rho(TrueModel,simulatedData)
            simAccData[i,:]=simulatedData
            abcCoordsx[t,i]=propx
            abcCoordsy[t,i]=propy 
            weightsDen=np.sum(weights[t-1,:]*scipy.stats.norm.pdf(propx,abcCoordsx[t-1,:],np.sqrt(2*varx))*scipy.stats.norm.pdf(propy,abcCoordsy[t-1,:],np.sqrt(2*vary)))
            weightsNum=scipy.stats.norm.pdf(propx,propCoord[0],10)*scipy.stats.norm.pdf(propy,propCoord[1],10)     
            weights[t,i]=weightsNum/weightsDen
    weights[t,:]=weights[t,:]/sum(weights[t,:])
print(totDraws)
abcOutput=np.column_stack((abcCoordsx[nIter-1,:],abcCoordsy[nIter-1,:]))
np.savetxt('abcPosteriorCoords_%d.dat'%(j), abcOutput)
importanceWeigths=weights[nIter-1,:]
np.savetxt('WeightsabcPosteriorCoords_%d.dat'%(j), importanceWeigths)
np.savetxt('simAccData_%d.dat'%(j), simAccData)
np.savetxt('TrueData_%d.dat'%(j), TrueModel)

In [17]:
###I pick randomly 10 elements for the analyses
elements=np.random.choice(np.shape(trueCoordMatrix)[0],1)
print(elements)

[3246]


In [18]:
import scipy.stats
###ABCPMC for all the events or a random selection among the possbilities
#for j in np.random.choice(np.shape(trueCoordMatrix)[0],10):
#for j in range(0,1): 
for j in elements:
    print(j)
    N=5000
    nIter=40
    quantile=0.85
    epsilon=4
    ###Importance weigths for the ABC-PMC
    weights=np.zeros((nIter,N))
    weights[0,:]=1/N
    ###abcCooord x
    abcCoordsx=np.zeros((nIter,N))
    ###abcCooord y
    abcCoordsy=np.zeros((nIter,N))
    ###Distance
    d=np.zeros((nIter,N)) 
    
    trueCoord = trueCoordMatrix[j]
    TrueModel=model(trueCoord[0], trueCoord[1], 500)
    naiveCoord=prior_mean(TrueModel)
    totDraws=0
    t=0
    for t in range(0,nIter):
        #print(t)
        if(t==0):
            for i in range(0,N):
                d[t,i]=epsilon+1
                while d[t,i] > epsilon: 
                    propCoord = priorFunction(naiveCoord)
                    ###Deterministic FM
                    #simulatedData = model(propCoord[0], propCoord[1], 500)
                    ###Stochastic FM, for now adding gaussian noise (mean=0,sd=1)
                    simulatedData = model(propCoord[0], propCoord[1], 500)+np.random.normal(0,1,np.shape(TrueModel)[0])
                    totDraws = totDraws+1
                    d[t,i] = rho(TrueModel,simulatedData)
                abcCoordsx[t,i]=propCoord[0]
                abcCoordsy[t,i]=propCoord[1]
        else:
            epsilon= np.percentile(d[t-1,],quantile*100)
            #print(epsilon)
            meanx=np.sum(abcCoordsx[t-1,:]*weights[t-1,:])
            varx=np.sum((abcCoordsx[t-1,:]-meanx)**2*weights[t-1,:])
            meany=np.sum(abcCoordsy[t-1,:]*weights[t-1,:])
            vary=np.sum((abcCoordsy[t-1,:]-meany)**2*weights[t-1,:])
            for i in range(0,N):
                d[t,i]=epsilon+1
                while d[t,i] > epsilon: 
                    sample=np.random.choice(N,1,p=weights[t-1,:])
                    propx0=abcCoordsx[t-1,sample]
                    propy0=abcCoordsy[t-1,sample]
                    prop=transfKernel(propx0,propy0,varx,vary)
                    propx=prop[0]
                    propy=prop[1]
                    ###Deterministic FM
                    #simulatedData = model(propx, propy, 500)
                    ###Stochastic FM, for now adding gaussian noise (mean=0,sd=1)
                    simulatedData = model(propx, propy, 500)+np.random.normal(0,1,np.shape(TrueModel)[0])
                    totDraws = totDraws+1
                    d[t,i] = rho(TrueModel,simulatedData)
                abcCoordsx[t,i]=propx
                abcCoordsy[t,i]=propy
                weightsDen=np.sum(weights[t-1,:]*scipy.stats.norm.pdf(propx,abcCoordsx[t-1,:],np.sqrt(2*varx))*scipy.stats.norm.pdf(propy,abcCoordsy[t-1,:],np.sqrt(2*vary)))
                weightsNum=scipy.stats.norm.pdf(propx,propCoord[0],10)*scipy.stats.norm.pdf(propy,propCoord[1],10)     
                weights[t,i]=weightsNum/weightsDen
        weights[t,:]=weights[t,:]/sum(weights[t,:])
    ###here we save the coordinated saved in the last iteration, and corresponding weights.
    abcOutput=np.column_stack((abcCoordsx[nIter-1,:],abcCoordsy[nIter-1,:]))
    np.savetxt('abcPosteriorCoords_%d.dat'%(j), abcOutput)
    importanceWeigths=weights[nIter-1,:]
    np.savetxt('WeightsabcPosteriorCoords_%d.dat'%(j), importanceWeigths)
    print(epsilon)

3246
0.875631205344
